In [8]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('data_motores.csv')
pd.set_option('display.max_columns', None)

In [9]:
TARIFA_T1 = (914.75 + 918.01)/2
TARIFA_T1

TARIFA_T2 = (798.52 + 801.84)/2
TARIFA_T2

800.1800000000001

In [10]:
#calculo de 11 meses, donde se trabaja continuamente 16 horas al dia de lunes a sabado
diasXmes        = 24
horasXdia       = 16
horasXmes       = diasXmes * horasXdia
meses           = 11
horasX11meses   = horasXmes * meses

#calculo para el mes de diciembre
diasDiciembre   = 15
horasDiciembre  = diasDiciembre * horasXdia

#total horas por año
TOTALHORAS     = horasX11meses + horasDiciembre

In [11]:
df1["Eficiencia (%)"]               = df1["Eficiencia (%)"]/100
df1["Factor de carga maximo"]       = df1["Factor de carga maximo"]/100
df1['Potencia Entrada (W)']         = (df1['Potencia (HP)'] * 746 * (df1['Factor de carga maximo'])) / (df1['Eficiencia (%)'])
df1['Potencia Salida (W)']  = (df1['Potencia (HP)'] * 746 * (df1['Factor de carga maximo']))
df1['Pérdidas Motor (W)']           = df1['Potencia Entrada (W)'] - df1['Potencia Salida (W)']
df1['Corriente a 220V (A)']         = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 220 * df1['Factor de potencia'])
df1['Corriente a 380V (A)']         = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 380 * df1['Factor de potencia'])
df1['Corriente a 220V (A) +25%']    = df1['Corriente a 220V (A)']*1.25
df1['Corriente a 380V (A) +25%']    = df1['Corriente a 380V (A)']*1.25
df1['Velocidad síncrona (RPM)']     = 3600

In [12]:
catalogo                            = pd.read_csv('catalogo_motores.csv')
catalogo['Potencia HP']             = catalogo['Potencia HP'].astype(int)
df2                                 = pd.merge(df1, catalogo[['Potencia HP', 'Velocidad rpm']], left_on='Potencia (HP)', right_on='Potencia HP', how='left')
df2.drop(columns=['Potencia HP'], inplace=True)
df2                                 = df2.rename(columns={'Velocidad rpm': 'Velocidad real (RPM)'})
df2['Deslizamiento (RPM)']          = df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)']
df2['Deslizamiento (%)']            = ((df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)']) / df2['Velocidad síncrona (RPM)'])*100
df2['Torque (N.m)']                 = (df2['Potencia (HP)'] * 746 * (df1['Factor de carga maximo'])) / ((df2['Velocidad real (RPM)'] * 2 * np.pi)/60)
df2['theta (°)']                    = np.degrees(np.arccos(df2['Factor de potencia']))
df2['Real Corriente 220V Normal']          = df2['Corriente a 220V (A)'] * np.cos(np.radians(df2['theta (°)']))
df2['Imaginaria Corriente 220V Normal']    = df2['Corriente a 220V (A)'] * np.sin(np.radians(df2['theta (°)']))
df2['Real Corriente 380V Normal']          = df2['Corriente a 380V (A)'] * np.cos(np.radians(df2['theta (°)']))
df2['Imaginaria Corriente 380V Normal']    = df2['Corriente a 380V (A)'] * np.sin(np.radians(df2['theta (°)']))
df2['Real Corriente 220V +25%']          = df2['Corriente a 220V (A) +25%'] * np.cos(np.radians(df2['theta (°)']))
df2['Imaginaria Corriente 220V +25%']    = df2['Corriente a 220V (A) +25%'] * np.sin(np.radians(df2['theta (°)']))
df2['Real Corriente 380V +25%']          = df2['Corriente a 380V (A) +25%'] * np.cos(np.radians(df2['theta (°)']))
df2['Imaginaria Corriente 380V +25%']    = df2['Corriente a 380V (A) +25%'] * np.sin(np.radians(df2['theta (°)']))
df2["kw/año"]                       = (df2["Potencia Entrada (W)"]/1000) * TOTALHORAS
df2["costo/año"]                    = df2["kw/año"] * TARIFA_T1

df2["cambio FC"]                    = np.where(df2["Factor de carga maximo"] < 0.8, "Si", "No")
df2["cambio Eficiencia"]            = np.where(df2["Eficiencia (%)"] < 0.9, "Si", "No")

In [13]:
df2.to_csv('data_motores_procesada.csv', index=False)

In [14]:
Corriente_Planta_1_220V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 220V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_2_220V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 220V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_3_220V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 220V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_4_220V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 220V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_5_220V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 220V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 220V Normal'].sum())**2)

Corriente_Planta_1_380V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 380V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_2_380V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 380V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_3_380V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 380V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_4_380V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 380V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_5_380V_Normal = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 380V Normal'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 380V Normal'].sum())**2)

Potencia_Planta_1_220 = 220*Corriente_Planta_1_220V_Normal
Potencia_Planta_2_220 = 220*Corriente_Planta_2_220V_Normal
Potencia_Planta_3_220 = 220*Corriente_Planta_3_220V_Normal
Potencia_Planta_4_220 = 220*Corriente_Planta_4_220V_Normal
Potencia_Planta_5_220 = 220*Corriente_Planta_5_220V_Normal

Potencia_Planta_1_380 = 380*Corriente_Planta_1_380V_Normal
Potencia_Planta_2_380 = 380*Corriente_Planta_2_380V_Normal
Potencia_Planta_3_380 = 380*Corriente_Planta_3_380V_Normal
Potencia_Planta_4_380 = 380*Corriente_Planta_4_380V_Normal
Potencia_Planta_5_380 = 380*Corriente_Planta_5_380V_Normal

# PARA 220V

Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [15]:
Corriente_Planta_1_220V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 220V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 1: {Corriente_Planta_1_220V} A')

Magnitud de corriente a 220V para los motores de la planta 1: 740.8613845652662 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [16]:
Corriente_Planta_2_220V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 220V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 2: {Corriente_Planta_2_220V} A')

Magnitud de corriente a 220V para los motores de la planta 2: 1055.846103440497 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [17]:
Corriente_Planta_3_220V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 220V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 3: {Corriente_Planta_3_220V} A')

Magnitud de corriente a 220V para los motores de la planta 3: 1184.1293915625126 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [18]:
Corriente_Planta_4_220V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 220V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 4: {Corriente_Planta_4_220V} A')

Magnitud de corriente a 220V para los motores de la planta 4: 774.7133890486778 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [19]:
Corriente_Planta_5_220V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 220V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 5: {Corriente_Planta_5_220V} A')

Magnitud de corriente a 220V para los motores de la planta 5: 1396.9715056603966 A


# PARA 380V

Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [20]:
Corriente_Planta_1_380V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 380V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 1: {Corriente_Planta_1_380V} A')

Magnitud de corriente a 380V para los motores de la planta 1: 428.91974895883834 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [21]:
Corriente_Planta_2_380V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 380V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 2: {Corriente_Planta_2_380V} A')

Magnitud de corriente a 380V para los motores de la planta 2: 611.2793230444981 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [22]:
Corriente_Planta_3_380V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 380V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 3: {Corriente_Planta_3_380V} A')

Magnitud de corriente a 380V para los motores de la planta 3: 685.5485951151388 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [23]:
Corriente_Planta_4_380V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 380V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 4: {Corriente_Planta_4_380V} A')

Magnitud de corriente a 380V para los motores de la planta 4: 448.51827787028714 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [24]:
Corriente_Planta_5_380V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 380V +25%'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 5: {Corriente_Planta_5_380V} A')

Magnitud de corriente a 380V para los motores de la planta 5: 808.7729769612822 A


Debido a que existen consumos auxiliares, se tiene que estos consumen 480kW, se puede partir de que poseen un factor de potencia de FP = 1, por lo tanto es pertinente calcular su consumo de corriente a 220V que es al voltaje al cual se encuentran conectados, suponiendo que son cargas puramente resistivas, estas no cuentan con desfase por lo tanto no poseen parte imaginaria, por lo que:

In [25]:
FP_aux = 1
Corriente_Auxiliares = 480000/(np.sqrt(3)*220*FP_aux)
print(f'Magnitud de corriente a 220V para los Auxiliares: {Corriente_Auxiliares} A')

Magnitud de corriente a 220V para los Auxiliares: 1259.6733145955473 A


## CAMBIO DE MOTORES ##

Dado un transformador de 13200/220V, según el documento de publicación de tarifas del mes de mayo de 2024, se tiene que para un transformador de este tipo se encuentra en el Nivel II de consumo por lo que la tarifa a pagar por poseer un contador antes del transformador es de:

Dado un transformador de 44000/220V, según el documento de publicación de tarifas del mes de mayo de 2024, se tiene que para un transformador de este tipo se encuentra en el Nivel III de consumo por lo que la tarifa a pagar por poseer un contador antes del transformador es de:

In [26]:
def actualizar_valores(row,df_filtro,potencia_seleccionada,potencia_real):
    potencia_real_02                            = potencia_real + potencia_real* 0.2
    row["Potencia (HP)"]                        = potencia_seleccionada
    row["Factor de carga maximo"]               = potencia_real_02 / potencia_seleccionada
    row["Eficiencia (%)"]                       = df_filtro.iloc[0]["Eficiencia %"] / 100
    row["Potencia Entrada (W)"]                 = (potencia_real * 746) / (row["Eficiencia (%)"])
    row['Potencia Salida (W)']                  = (potencia_real * 746)
    row["Corriente a 220V (A)"]                 = row["Potencia Entrada (W)"] / (np.sqrt(3) * 220 * row["Factor de potencia"])
    row['Corriente a 380V (A)']                 = row['Potencia Entrada (W)'] / (np.sqrt(3) * 380 * row['Factor de potencia'])
    row['Pérdidas Motor (W)']                   = row['Potencia Entrada (W)'] - row['Potencia Salida (W)']
    row['Corriente a 220V (A) +25%']            = row['Corriente a 220V (A)']*1.25
    row['Corriente a 380V (A) +25%']            = row['Corriente a 380V (A)']*1.25
    row["Velocidad real (RPM)"]                 = df_filtro.iloc[0]["Velocidad rpm"]
    row['Deslizamiento (RPM)']                  = row['Velocidad síncrona (RPM)'] - row['Velocidad real (RPM)']
    row['Deslizamiento (%)']                    = ((row['Velocidad síncrona (RPM)'] - row['Velocidad real (RPM)']) / row['Velocidad síncrona (RPM)'])*100
    row['Torque (N.m)']                         = (row['Potencia (HP)'] * 746 * (row['Factor de carga maximo'])) / ((row['Velocidad real (RPM)'] * 2 * np.pi)/60)
    row['theta (°)']                            = np.degrees(np.arccos(row['Factor de potencia']))
    row['Real Corriente 220V Normal']           = row['Corriente a 220V (A)'] * np.cos(np.radians(row['theta (°)']))
    row['Imaginaria Corriente 220V Normal']     = row['Corriente a 220V (A)'] * np.sin(np.radians(row['theta (°)']))
    row['Real Corriente 380V Normal']           = row['Corriente a 380V (A)'] * np.cos(np.radians(row['theta (°)']))
    row['Imaginaria Corriente 380V Normal']     = row['Corriente a 380V (A)'] * np.sin(np.radians(row['theta (°)']))
    row['Real Corriente 220V +25%']             = row['Corriente a 220V (A) +25%'] * np.cos(np.radians(row['theta (°)']))
    row['Imaginaria Corriente 220V +25%']       = row['Corriente a 220V (A) +25%'] * np.sin(np.radians(row['theta (°)']))
    row['Real Corriente 380V +25%']             = row['Corriente a 380V (A) +25%'] * np.cos(np.radians(row['theta (°)']))
    row['Imaginaria Corriente 380V +25%']       = row['Corriente a 380V (A) +25%'] * np.sin(np.radians(row['theta (°)']))
    row["kw/año"]                               = (row["Potencia Entrada (W)"]/1000) * TOTALHORAS
    row["costo/año"]                            = row["kw/año"] * TARIFA_T1
    return row


In [27]:
df_catalogo = pd.read_csv("catalogo_motores_completo.csv")
df_catalogo = df_catalogo[df_catalogo["rpm"]==3600]
def cambio(row):

    cambio_fc         = False
    cambio_eficiencia = False
    #se debe verificar si el cambio es por factor de carga o factor de potencia
    if  row['cambio FC'] == 'Si':
        potencia_real     = row['Potencia (HP)'] * row["Factor de carga maximo"]

        #calculo del valor, mas un 20%
        potencia_real_02  = potencia_real + potencia_real* 0.2

        #buscamos en el catalogo los motores mayores al valor y escogemos el primero
        df_filtro               = df_catalogo[df_catalogo["Potencia HP"] >= potencia_real_02].sort_values(by="Potencia HP", ascending=True).copy()
        potencia_seleccionada   = df_filtro.iloc[0]["Potencia HP"]
        # print(row['Potencia (HP)'],row["Factor de carga maximo"],potencia_real,potencia_real_02,potencia_seleccionada)

        if potencia_seleccionada == row['Potencia (HP)']:
            # print("no hay cambio debido a que el motor que se encontro, es del mismo tamaño que el que se tiene")
            cambio_fc = False
        else:
            # print("se encontro un motor de menor tamaño al que se le puede aplicar el cambio")
            cambio_fc = True

            #se procede a actualizar el resto de calculos para este valor
            row = actualizar_valores(row,df_filtro,potencia_seleccionada,potencia_real)


    #si no hay cambio por factor de carga, se verifica si lo hay por eficiencia
    if cambio_fc == False and row['cambio Eficiencia'] == 'Si':
          df_filtro     = df_catalogo[df_catalogo["Potencia HP"] == row["Potencia (HP)"]].copy()
          #row,df_filtro,potencia_seleccionada,potencia_real
          row           = actualizar_valores(row,df_filtro,row["Potencia (HP)"],row["Potencia (HP)"]*row["Factor de carga maximo"])
          cambio_eficiencia = True
    else:
        cambio_eficiencia = False


    if cambio_eficiencia == True or cambio_fc == True:
        row["cambio"] = "Si"
    else:
        row["cambio"] = "No"

    return row



    #factor de carga, potencia_hp, potencia_actual
    # costoXaño = (potencia_actual / 1000) * TOTALHORAS * TARIFA_T1 #Reemplazar TARIFA_T1 por TARIFA_T2 si se desea calcular con la tarifa T2
    # df_filtro                           = df_catalogo[(df_catalogo["Potencia HP"] == potencia_hp) & (df_catalogo["rpm"] == velocidad_sincronica)].copy()
    # df_filtro['Potencia Entrada (W)']   = (df_filtro ['Potencia HP'] * 746 * (fc)) / (df_filtro ['Eficiencia %']/100)
    # df_filtro["kw/año"]                 = (df_filtro["Potencia Entrada (W)"]/1000) * TOTALHORAS
    # df_filtro["costo/año"]              = df_filtro["kw/año"] * TARIFA_T1
    # df_filtro["costo_anterior"]         = costoXaño
    # df_filtro["ahorroXmes"]             = (df_filtro["costo_anterior"] - df_filtro["costo/año"]) / 12
    # df_filtro["tiempo_retribucion"]     = df_filtro["costo_Pesos"] / df_filtro["ahorroXmes"]


In [28]:
df2[(df2["cambio FC"] == "Si") | (df2["cambio Eficiencia"] == "Si")].shape


(23, 32)

In [29]:
df3 = df2.copy()
df3 = df3.apply(cambio, axis=1)
orden = df2.columns.to_list()
orden.append('cambio')
df3 = df3[orden]

In [30]:
Corriente_Planta_1_220V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 1']['Real Corriente 220V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 1']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_2_220V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 2']['Real Corriente 220V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 2']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_3_220V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 3']['Real Corriente 220V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 3']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_4_220V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 4']['Real Corriente 220V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 4']['Imaginaria Corriente 220V Normal'].sum())**2)
Corriente_Planta_5_220V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 5']['Real Corriente 220V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 5']['Imaginaria Corriente 220V Normal'].sum())**2)

Corriente_Planta_1_380V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 1']['Real Corriente 380V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 1']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_2_380V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 2']['Real Corriente 380V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 2']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_3_380V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 3']['Real Corriente 380V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 3']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_4_380V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 4']['Real Corriente 380V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 4']['Imaginaria Corriente 380V Normal'].sum())**2)
Corriente_Planta_5_380V_Normal_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 5']['Real Corriente 380V Normal'].sum())**2 + (df3[df3['Planta'] == 'Planta 5']['Imaginaria Corriente 380V Normal'].sum())**2)

Potencia_Planta_1_220_cambio = 220*Corriente_Planta_1_220V_Normal_cambio
Potencia_Planta_2_220_cambio = 220*Corriente_Planta_2_220V_Normal_cambio
Potencia_Planta_3_220_cambio = 220*Corriente_Planta_3_220V_Normal_cambio
Potencia_Planta_4_220_cambio = 220*Corriente_Planta_4_220V_Normal_cambio
Potencia_Planta_5_220_cambio = 220*Corriente_Planta_5_220V_Normal_cambio

Potencia_Planta_1_380_cambio = 380*Corriente_Planta_1_380V_Normal_cambio
Potencia_Planta_2_380_cambio = 380*Corriente_Planta_2_380V_Normal_cambio
Potencia_Planta_3_380_cambio = 380*Corriente_Planta_3_380V_Normal_cambio
Potencia_Planta_4_380_cambio = 380*Corriente_Planta_4_380V_Normal_cambio
Potencia_Planta_5_380_cambio = 380*Corriente_Planta_5_380V_Normal_cambio

In [31]:
df2.iloc[0]

Planta                                      Planta 1
Distancia                                        200
Motor                                              1
Potencia (HP)                                     30
Eficiencia (%)                                   0.7
Factor de carga maximo                           0.8
Factor de potencia                              0.74
Potencia Entrada (W)                    25577.142857
Corriente a 220V (A)                       90.706206
Corriente a 380V (A)                       52.514119
Potencia Salida (W)                          17904.0
Pérdidas Motor (W)                       7673.142857
Corriente a 220V (A) +25%                 113.382757
Corriente a 380V (A) +25%                  65.642649
Velocidad síncrona (RPM)                        3600
Velocidad real (RPM)                            3540
Deslizamiento (RPM)                               60
Deslizamiento (%)                           1.666667
Torque (N.m)                               48.

In [32]:
df3.iloc[0]

Planta                                     Planta 1
Distancia                                       200
Motor                                             1
Potencia (HP)                                  30.0
Eficiencia (%)                                0.917
Factor de carga maximo                         0.96
Factor de potencia                             0.74
Potencia Entrada (W)                   19524.536532
Corriente a 220V (A)                      69.241379
Corriente a 380V (A)                      40.087114
Potencia Salida (W)                         17904.0
Pérdidas Motor (W)                      1620.536532
Corriente a 220V (A) +25%                 86.551723
Corriente a 380V (A) +25%                 50.108892
Velocidad síncrona (RPM)                       3600
Velocidad real (RPM)                           3540
Deslizamiento (RPM)                              60
Deslizamiento (%)                          1.666667
Torque (N.m)                              57.956138
theta (°)   

Según [Tabla verde, luego paso el link], dado un calibre de 500 mm2 se tiene una resistencia DC de Ohm/km para un cable de cobre, se tiene que posee una resistencia de 0.0366 Ohm por lo que si se requiere de una corriente determinada para alimentar cada planta a 220V o 380V es necesario aplicar una regla de 3 compuesta para extraer el valor de la resistencia de pérdida a partir de https://www.gt-engineering.it/en/technical-standards/ul-and-nfpa-standards/ul-508a/cable-ampacity-ul-508a/, también es de anotar que es posible alimentar con un solo cable a partir de suma de corrientes, es decir, con corrientes menores por cada cable que se use para complementar la corriente objetivo a usar.

# PARA 220V

Grosor del cable sobre la Planta 1, a 220V:

In [33]:
print(f'La corriente necesaria es de {Corriente_Planta_1_220V} A para la planta 1, por lo que se necesita de dos cables que soporten 380 A de cobre, el cual corresponde a un valor de 500 kcmil.')

R_km_P1_220 = (500/253)*0.0366
print(f'El grosor del cable necesario para 380 A es de 253 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P1_220} Ohm')

Resistencia_Final_P1_220 = (200*R_km_P1_220)/1000
print(f'Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P1_220} Ohm')

La corriente necesaria es de 740.8613845652662 A para la planta 1, por lo que se necesita de dos cables que soporten 380 A de cobre, el cual corresponde a un valor de 500 kcmil.
El grosor del cable necesario para 380 A es de 253 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.07233201581027668 Ohm
Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.014466403162055335 Ohm


In [34]:
Paralelo_R_km_P1_220 = Resistencia_Final_P1_220/2
print(Paralelo_R_km_P1_220)
Perdida_P1_220 = Corriente_Planta_1_220V_Normal**2 * (Paralelo_R_km_P1_220)
print(Corriente_Planta_1_220V_Normal)
print(f'La pérdida asociada a la corriente a 220V para la planta 1 de una sola línea son de {Perdida_P1_220} W')
Perdida_P1_220_Tri = 3*Perdida_P1_220
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P1_220_Tri} W')

0.007233201581027668
592.689107652213
La pérdida asociada a la corriente a 220V para la planta 1 de una sola línea son de 2540.8817879174903 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 7622.645363752471 W


Grosor del cable sobre la Planta 2, a 220V:

In [35]:
print(f'La corriente necesaria es de {Corriente_Planta_2_220V} A para la planta 1, por lo que se necesita de dos cables que soporten 545 A de cobre, el cual corresponde a un valor de 1000 kcmil.')

R_km_P2_220 = (500/506)*0.0366
print(f'El grosor del cable necesario para 545 A es de 506 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P2_220} Ohm')

Resistencia_Final_P2_220 = (300*R_km_P2_220)/1000
print(f'Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P2_220} Ohm')

La corriente necesaria es de 1055.846103440497 A para la planta 1, por lo que se necesita de dos cables que soporten 545 A de cobre, el cual corresponde a un valor de 1000 kcmil.
El grosor del cable necesario para 545 A es de 506 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.03616600790513834 Ohm
Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.010849802371541501 Ohm


In [36]:
Paralelo_R_km_P2_220 = Resistencia_Final_P2_220/2
Perdida_P2_220 = Corriente_Planta_2_220V_Normal**2 * (Paralelo_R_km_P2_220)
print(f'La pérdida asociada a la corriente a 220V para la planta 2 son de {Perdida_P2_220} W')
Perdida_P2_220_Tri = 3*Perdida_P2_220
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P2_220_Tri} W')

La pérdida asociada a la corriente a 220V para la planta 2 son de 3870.553269809416 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 11611.659809428249 W


Grosor del cable sobre la Planta 3, a 220V:

In [37]:
print(f'La corriente necesaria es de {Corriente_Planta_3_220V} A para la planta 1, por lo que se necesita de dos cables que soporten 625 A de cobre, el cual corresponde a un valor de 1500 kcmil.')

R_km_P3_220 = (500/887)*0.0366
print(f'El grosor del cable necesario para 625 A es de 887 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P3_220} Ohm')

Resistencia_Final_P3_220 = (350*R_km_P3_220)/1000
print(f'Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P3_220} Ohm')

La corriente necesaria es de 1184.1293915625126 A para la planta 1, por lo que se necesita de dos cables que soporten 625 A de cobre, el cual corresponde a un valor de 1500 kcmil.
El grosor del cable necesario para 625 A es de 887 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.020631341600901916 Ohm
Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.00722096956031567 Ohm


In [38]:
Paralelo_R_km_P3_220 = Resistencia_Final_P3_220/2
Perdida_P3_220 = Corriente_Planta_3_220V_Normal**2 * (Paralelo_R_km_P3_220)
print(f'La pérdida asociada a la corriente a 220V para la planta 3 son de {Perdida_P3_220} W')
Perdida_P3_220_Tri = 3*Perdida_P3_220
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P3_220_Tri} W')

La pérdida asociada a la corriente a 220V para la planta 3 son de 3239.991079770167 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 9719.9732393105 W


Grosor del cable sobre la Planta 4, a 220V:

In [39]:
print(f'La corriente necesaria es de {Corriente_Planta_4_220V} A para la planta 1, por lo que se necesita de dos cables que soporten 420 A de cobre, el cual corresponde a un valor de 600 kcmil.')

R_km_P4_220 = (500/304)*0.0366
print(f'El grosor del cable necesario para 420 A es de 304 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P4_220} Ohm')

Resistencia_Final_P4_220 = (300*R_km_P4_220)/1000
print(f'Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P4_220} Ohm')

La corriente necesaria es de 774.7133890486778 A para la planta 1, por lo que se necesita de dos cables que soporten 420 A de cobre, el cual corresponde a un valor de 600 kcmil.
El grosor del cable necesario para 420 A es de 304 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.06019736842105263 Ohm
Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.018059210526315792 Ohm


In [40]:
Paralelo_R_km_P4_220 = Resistencia_Final_P4_220/2
Perdida_P4_220 = Corriente_Planta_4_220V_Normal**2 * (Paralelo_R_km_P4_220)
print(f'La pérdida asociada a la corriente a 220V para la planta 4 son de {Perdida_P4_220} W')
Perdida_P4_220_Tri = 3*Perdida_P4_220
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P4_220_Tri} W')

La pérdida asociada a la corriente a 220V para la planta 4 son de 3468.4134579898637 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 10405.240373969591 W


Grosor del cable sobre la Planta 5, a 220V:

In [41]:
print(f'La corriente necesaria es de {Corriente_Planta_5_220V} A para la planta 1, por lo que se necesita de tres cables que soporten 475 A de cobre, el cual corresponde a un valor de 750 kcmil.')

R_km_P5_220 = (500/380)*0.0366
print(f'El grosor del cable necesario para 475 A es de 380 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P5_220} Ohm')

Resistencia_Final_P5_220 = (500*R_km_P5_220)/1000
print(f'Ahora bien, la Planta 5 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P5_220} Ohm')

La corriente necesaria es de 1396.9715056603966 A para la planta 1, por lo que se necesita de tres cables que soporten 475 A de cobre, el cual corresponde a un valor de 750 kcmil.
El grosor del cable necesario para 475 A es de 380 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.04815789473684211 Ohm
Ahora bien, la Planta 5 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.024078947368421054 Ohm


In [42]:
Paralelo_R_km_P5_220 = Resistencia_Final_P5_220/3
Perdida_P5_220 = Corriente_Planta_5_220V_Normal**2 * (Paralelo_R_km_P5_220)
print(f'La pérdida asociada a la corriente a 220V para la planta 5 son de {Perdida_P5_220} W')
Perdida_P5_220_Tri = 3*Perdida_P5_220
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P5_220_Tri} W')

La pérdida asociada a la corriente a 220V para la planta 5 son de 10024.698328021188 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 30074.094984063566 W


# PARA 380V

Grosor del cable sobre la Planta 1, a 380V:

In [43]:
print(f'La corriente necesaria es de {Corriente_Planta_1_380V} A para la planta 1, por lo que se necesita de un solo cable que soporte 445 A de aluminio, el cual corresponde a un valor de 1000 kcmil.')

R_km_P1_380 = (500/506)*0.061
print(f'El grosor del cable necesario para 445 A es de 506 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P1_380} Ohm')

Resistencia_Final_P1_380 = (200*R_km_P1_380)/1000
print(f'Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P1_380} Ohm')

La corriente necesaria es de 428.91974895883834 A para la planta 1, por lo que se necesita de un solo cable que soporte 445 A de aluminio, el cual corresponde a un valor de 1000 kcmil.
El grosor del cable necesario para 445 A es de 506 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.06027667984189723 Ohm
Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.012055335968379446 Ohm


In [44]:
Perdida_P1_380 = Corriente_Planta_1_380V_Normal**2 * (Resistencia_Final_P1_380)
print(f'La pérdida asociada a la corriente a 380V para la planta 1 son de {Perdida_P1_380} W')
Perdida_P1_380_Tri = 3*Perdida_P1_380
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P1_380_Tri} W')

La pérdida asociada a la corriente a 380V para la planta 1 son de 1419.4214974054307 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 4258.264492216293 W


Grosor del cable sobre la Planta 2, a 380V:

In [112]:
print(f'La corriente necesaria es de {Corriente_Planta_2_380V} A para la planta 1, por lo que se necesita de un solo cable que soporte 625 A de cobre, el cual corresponde a un valor de 1500 kcmil.')

R_km_P2_380 = (500/760)*0.0366
print(f'El grosor del cable necesario para 625 A es de 760 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P2_380} Ohm')

Resistencia_Final_P2_380 = (300*R_km_P2_380)/1000
print(f'Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P2_380} Ohm')

La corriente necesaria es de 611.2793230444981 A para la planta 1, por lo que se necesita de un solo cable que soporte 625 A de cobre, el cual corresponde a un valor de 1500 kcmil.
El grosor del cable necesario para 625 A es de 760 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.024078947368421054 Ohm
Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.007223684210526316 Ohm


In [46]:
Perdida_P2_380 = Corriente_Planta_2_380V_Normal**2 * (Resistencia_Final_P2_380)
print(f'La pérdida asociada a la corriente a 380V para la planta 2 son de {Perdida_P2_380} W')
Perdida_P2_380_Tri = Perdida_P2_380
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P2_380_Tri} W')

La pérdida asociada a la corriente a 380V para la planta 2 son de 1727.5003243720023 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 1727.5003243720023 W


Grosor del cable sobre la Planta 3, a 380V:

In [47]:
print(f'La corriente necesaria es de {Corriente_Planta_3_380V} A para la planta 1, por lo que se necesita de dos cables que soporte 375 A de aluminio, el cual corresponde a un valor de 700 kcmil.')

R_km_P3_380 = (500/355)*0.061
print(f'El grosor del cable necesario para 375 A es de 355 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P3_380} Ohm')

Resistencia_Final_P3_380 = (350*R_km_P3_380)/1000
print(f'Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P3_380} Ohm')

La corriente necesaria es de 685.5485951151388 A para la planta 1, por lo que se necesita de dos cables que soporte 375 A de aluminio, el cual corresponde a un valor de 700 kcmil.
El grosor del cable necesario para 375 A es de 355 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.08591549295774648 Ohm
Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.030070422535211267 Ohm


In [48]:
Paralelo_R_km_P3_380 = Resistencia_Final_P3_220/2
Perdida_P3_380 = Corriente_Planta_3_380V_Normal**2 * (Paralelo_R_km_P3_380)
print(f'La pérdida asociada a la corriente a 220V para la planta 5 son de {Perdida_P3_380} W')
Perdida_P3_380_Tri = 3*Perdida_P3_380
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P3_380_Tri} W')

La pérdida asociada a la corriente a 220V para la planta 5 son de 1085.9803896182552 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 3257.9411688547657 W


Grosor del cable sobre la Planta 4, a 380V:

In [49]:
print(f'La corriente necesaria es de {Corriente_Planta_4_380V} A para la planta 1, por lo que se necesita de un solo cable que soporte 460 A de cobre, el cual corresponde a un valor de 750 kcmil.')

R_km_P4_380 = (500/355)*0.0366
print(f'El grosor del cable necesario para 460 A es de 355 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P4_380} Ohm')

Resistencia_Final_P4_380 = (300*R_km_P4_380)/1000
print(f'Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P4_380} Ohm')

La corriente necesaria es de 448.51827787028714 A para la planta 1, por lo que se necesita de un solo cable que soporte 460 A de cobre, el cual corresponde a un valor de 750 kcmil.
El grosor del cable necesario para 460 A es de 355 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.05154929577464788 Ohm
Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.015464788732394364 Ohm


In [50]:
Perdida_P4_380 = Corriente_Planta_4_380V_Normal**2 * (Resistencia_Final_P4_380)
print(f'La pérdida asociada a la corriente a 380V para la planta 2 son de {Perdida_P4_380} W')
Perdida_P4_380_Tri = 3*Perdida_P4_380
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P4_380_Tri} W')

La pérdida asociada a la corriente a 380V para la planta 2 son de 1991.0595862619343 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 5973.178758785803 W


Grosor del cable sobre la Planta 5, a 380V:

In [51]:
print(f'La corriente necesaria es de {Corriente_Planta_5_380V} A para la planta 1, por lo que se necesita de dos cables que soporte 420 A de cobre, el cual corresponde a un valor de 600 kcmil.')

R_km_P5_380 = (500/420)*0.0366
print(f'El grosor del cable necesario para 420 A es de 304 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P5_380} Ohm')

Resistencia_Final_P5_380 = (500*R_km_P5_380)/1000
print(f'Ahora bien, la Planta 3 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P5_380} Ohm')

La corriente necesaria es de 808.7729769612822 A para la planta 1, por lo que se necesita de dos cables que soporte 420 A de cobre, el cual corresponde a un valor de 600 kcmil.
El grosor del cable necesario para 420 A es de 304 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.04357142857142857 Ohm
Ahora bien, la Planta 3 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.021785714285714287 Ohm


In [52]:
Paralelo_R_km_P5_380 = Resistencia_Final_P5_380/2
Perdida_P5_380 = Corriente_Planta_5_380V_Normal**2 * (Paralelo_R_km_P5_380)
print(f'La pérdida asociada a la corriente a 220V para la planta 5 son de {Perdida_P5_380} W')
Perdida_P5_380_Tri = 3*Perdida_P5_380
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P5_380_Tri} W')

La pérdida asociada a la corriente a 220V para la planta 5 son de 4560.107134175054 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 13680.321402525162 W


Esta ecuación es para cuando es a 220 o 380

PotenciaEntradaTransformador_sinAuxiliares = PotenciaEntradaTransformador_sinAuxiliares + (PotenciaPlanta(i) + PerdidasPlanta(i))/EficienciaTransformador

PotenciaEntradaTransformador = PotenciaEntradaTransformador_sinAuxiliares + Potencia_Auxiliares

CostoTotalAnual = PotenciaEntradaTransformador * DíasTrabajo_Año * Precio(Nivel 2 o 3)


In [53]:
PerdidasXPlanta_Trifasica_220 = [Perdida_P1_220_Tri, Perdida_P2_220_Tri, Perdida_P3_220_Tri, Perdida_P4_220_Tri, Perdida_P5_220_Tri]
PotenciaXPlanta_220 = [Potencia_Planta_1_220, Potencia_Planta_2_220, Potencia_Planta_3_220, Potencia_Planta_4_220, Potencia_Planta_5_220]

PotenciaEntradaTransformador_sinAuxiliares_220 = 0
Eficiencia_Transformador = 0.9
Potencia_Auxiliares = 480000

for i in range(len(PerdidasXPlanta_Trifasica_220)):
    PotenciaEntradaTransformador_sinAuxiliares_220 += (PerdidasXPlanta_Trifasica_220[i] + PotenciaXPlanta_220[i])/Eficiencia_Transformador

PotenciaEntradaTransformador_220 = PotenciaEntradaTransformador_sinAuxiliares_220 + Potencia_Auxiliares

print(f'La potencia de entrada del transformador a 220V es de {PotenciaEntradaTransformador_220:,} W')

La potencia de entrada del transformador a 220V es de 1,564,752.717825931 W


In [54]:
PerdidasXPlanta_Trifasica_380 = [Perdida_P1_380_Tri, Perdida_P2_380_Tri, Perdida_P3_380_Tri, Perdida_P4_380_Tri, Perdida_P5_380_Tri]
PotenciaXPlanta_380 = [Potencia_Planta_1_380, Potencia_Planta_2_380, Potencia_Planta_3_380, Potencia_Planta_4_380, Potencia_Planta_5_380]

PotenciaEntradaTransformador_sinAuxiliares_380 = 0
Eficiencia_Transformador = 0.9
Potencia_Auxiliares = 480000

for i in range(len(PerdidasXPlanta_Trifasica_380)):
    PotenciaEntradaTransformador_sinAuxiliares_380 += (PerdidasXPlanta_Trifasica_380[i] + PotenciaXPlanta_380[i])/Eficiencia_Transformador

PotenciaEntradaTransformador_380 = PotenciaEntradaTransformador_sinAuxiliares_380 + Potencia_Auxiliares

print(f'La potencia de entrada del transformador a 380V es de {PotenciaEntradaTransformador_380:,} W')

La potencia de entrada del transformador a 380V es de 1,519,712.2649106306 W


In [55]:
CostoTotal_220_13200 = (PotenciaEntradaTransformador_220/1000) * TOTALHORAS * TARIFA_T1
CostoTotal_380_13200 = (PotenciaEntradaTransformador_380/1000) * TOTALHORAS * TARIFA_T1
print(f'El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 13200/220 es de {CostoTotal_220_13200:,} pesos')
print(f'El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 13200/220 es de {CostoTotal_380_13200:,} pesos')

CostoTotal_220_44000 = (PotenciaEntradaTransformador_220/1000) * TOTALHORAS * TARIFA_T2
CostoTotal_380_44000 = (PotenciaEntradaTransformador_380/1000) * TOTALHORAS * TARIFA_T2
print(f'El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 44000/220 es de {CostoTotal_220_44000:,} pesos')
print(f'El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 44000/220 es de {CostoTotal_380_44000:,} pesos')

El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 13200/220 es de 6,400,965,738.585762 pesos
El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 13200/220 es de 6,216,717,842.62314 pesos
El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 44000/220 es de 5,589,302,216.003793 pesos
El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 44000/220 es de 5,428,417,559.647945 pesos


In [70]:
df3

,Planta,Distancia,Motor,Potencia (HP),Eficiencia (%),Factor de carga maximo,Factor de potencia,Potencia Entrada (W),Corriente a 220V (A),Corriente a 380V (A),Potencia Salida (W),Pérdidas Motor (W),Corriente a 220V (A) +25%,Corriente a 380V (A) +25%,Velocidad síncrona (RPM),Velocidad real (RPM),Deslizamiento (RPM),Deslizamiento (%),Torque (N.m),theta (°),Real Corriente 220V Normal,Imaginaria Corriente 220V Normal,Real Corriente 380V Normal,Imaginaria Corriente 380V Normal,Real Corriente 220V +25%,Imaginaria Corriente 220V +25%,Real Corriente 380V +25%,Imaginaria Corriente 380V +25%,kw/año,costo/año,cambio FC,cambio Eficiencia,cambio
0,Planta 1,200,1,30.0,0.917,0.960,0.74,19524.536532,69.241379,40.087114,17904.0,1620.536532,86.551723,50.108892,3600,3540,60,1.666667,57.956138,42.268584,51.238620,46.572227,29.664464,26.962868,64.048275,58.215283,37.080580,33.703585,87157.531080,7.986942e+07,No,Si,Si
1,Planta 2,300,1,40.0,0.924,0.960,0.82,25835.497835,82.683656,47.869485,23872.0,1963.497835,103.354571,59.836857,3600,3565,35,0.972222,76.732951,34.915206,67.800598,47.325109,39.252978,27.398747,84.750748,59.156386,49.066222,34.248434,115329.662338,1.056858e+08,No,Si,Si
2,Planta 3,350,1,50.0,0.930,0.864,0.78,28877.419355,97.158426,56.249615,26856.0,2021.419355,121.448032,70.312019,3600,3570,30,0.833333,86.203667,38.739425,75.783572,60.799752,43.874700,35.199857,94.729465,75.999690,54.843374,43.999821,128908.800000,1.181294e+08,Si,Si,Si
3,Planta 4,300,1,25.0,0.917,0.864,0.78,14643.402399,49.267904,28.523523,13428.0,1215.402399,61.584880,35.654404,3600,3545,55,1.527778,43.405796,38.739425,38.428965,30.830845,22.248348,17.849437,48.036206,38.538556,27.810435,22.311796,65368.148310,5.990206e+07,Si,Si,Si
4,Planta 5,500,1,75.0,0.936,0.960,0.81,47820.512821,154.933703,89.698460,44760.0,3060.512821,193.667129,112.123075,3600,3577,23,0.638889,143.391618,35.904069,125.496300,90.857753,72.655752,52.601857,156.870375,113.572191,90.819691,65.752321,213470.769231,1.956203e+08,Si,No,Si
5,Planta 1,200,2,50.0,0.930,0.960,0.75,32086.021505,112.271959,64.999555,29840.0,2246.021505,140.339948,81.249444,3600,3570,30,0.833333,95.781852,41.409622,84.203969,74.260920,48.749666,42.993164,105.254961,92.826150,60.937083,53.741456,143232.000000,1.312549e+08,No,Si,Si
6,Planta 2,300,2,125.0,0.950,0.960,0.83,78526.315789,248.286909,143.745053,74600.0,3926.315789,310.358636,179.681316,3600,3580,20,0.555556,238.785763,33.901262,206.078134,138.485348,119.308394,80.175728,257.597668,173.106685,149.135492,100.219660,350541.473684,3.212292e+08,No,Si,Si
7,Planta 3,350,2,150.0,0.900,0.800,0.79,99466.666667,330.420637,191.296158,89520.0,9946.666667,413.025797,239.120198,3600,3580,20,0.555556,238.785763,37.814489,261.032304,202.583154,151.123965,117.284984,326.290379,253.228942,188.904956,146.606230,444019.200000,4.068903e+08,No,No,No
8,Planta 4,300,2,30.0,0.917,0.960,0.79,19524.536532,64.859013,37.549955,17904.0,1620.536532,81.073766,46.937443,3600,3540,60,1.666667,57.956138,37.814489,51.238620,39.765505,29.664464,23.022134,64.048275,49.706881,37.080580,28.777668,87157.531080,7.986942e+07,Si,Si,Si
9,Planta 5,500,2,200.0,0.954,0.900,0.82,117295.597484,375.391601,217.331979,111900.0,5395.597484,469.239501,271.664974,3600,3582,18,0.500000,357.978656,34.915206,307.821113,214.860458,178.212223,124.392897,384.776391,268.575573,222.765279,155.491121,523607.547170,4.798235e+08,Si,No,Si


# PARA 220V CAMBIO DE MOTOR

Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [71]:
Corriente_Planta_1_220V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 1']['Real Corriente 220V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 1']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 1: {Corriente_Planta_1_220V_cambio} A')

Magnitud de corriente a 220V para los motores de la planta 1: 654.1128222688013 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [72]:
Corriente_Planta_2_220V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 2']['Real Corriente 220V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 2']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 2: {Corriente_Planta_2_220V_cambio} A')

Magnitud de corriente a 220V para los motores de la planta 2: 988.3047326449503 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [76]:
Corriente_Planta_3_220V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 3']['Real Corriente 220V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 3']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 3: {Corriente_Planta_3_220V_cambio} A')

Magnitud de corriente a 220V para los motores de la planta 3: 1128.8194722091384 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [77]:
Corriente_Planta_4_220V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 4']['Real Corriente 220V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 4']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 4: {Corriente_Planta_4_220V_cambio} A')

Magnitud de corriente a 220V para los motores de la planta 4: 737.4429373393467 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [78]:
Corriente_Planta_5_220V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 5']['Real Corriente 220V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 5']['Imaginaria Corriente 220V +25%'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 5: {Corriente_Planta_5_220V_cambio} A')

Magnitud de corriente a 220V para los motores de la planta 5: 1364.6728177373398 A


# PARA 380V CAMBIO DE MOTOR

Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [79]:
Corriente_Planta_1_380V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 1']['Real Corriente 380V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 1']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 1: {Corriente_Planta_1_380V_cambio} A')

Magnitud de corriente a 380V para los motores de la planta 1: 378.69689710299025 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [82]:
Corriente_Planta_2_380V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 2']['Real Corriente 380V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 2']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 2: {Corriente_Planta_2_380V_cambio} A')

Magnitud de corriente a 380V para los motores de la planta 2: 572.1764241628659 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [81]:
Corriente_Planta_3_380V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 3']['Real Corriente 380V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 3']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 3: {Corriente_Planta_3_380V_cambio} A')

Magnitud de corriente a 380V para los motores de la planta 3: 653.5270628579223 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [84]:
Corriente_Planta_4_380V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 4']['Real Corriente 380V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 4']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 4: {Corriente_Planta_4_380V_cambio} A')

Magnitud de corriente a 380V para los motores de la planta 4: 426.9406479333061 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [85]:
Corriente_Planta_5_380V_cambio = np.sqrt((df3[df3['Planta'] == 'Planta 5']['Real Corriente 380V +25%'].sum())**2 + (df3[df3['Planta'] == 'Planta 5']['Imaginaria Corriente 380V +25%'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 5: {Corriente_Planta_5_380V_cambio} A')

Magnitud de corriente a 380V para los motores de la planta 5: 790.0737365847755 A


Debido a que existen consumos auxiliares, se tiene que estos consumen 480kW, se puede partir de que poseen un factor de potencia de FP = 1, por lo tanto es pertinente calcular su consumo de corriente a 220V que es al voltaje al cual se encuentran conectados, suponiendo que son cargas puramente resistivas, estas no cuentan con desfase por lo tanto no poseen parte imaginaria, por lo que:

In [67]:
FP_aux = 1
Corriente_Auxiliares = 480000/(np.sqrt(3)*220*FP_aux)
print(f'Magnitud de corriente a 220V para los Auxiliares: {Corriente_Auxiliares} A')

Magnitud de corriente a 220V para los Auxiliares: 1259.6733145955473 A


# DIMENSIONAMIENTO DEL CABLE CUANDO SE CAMBIAN LOS MOTORES

Según [Tabla verde, luego paso el link], dado un calibre de 500 mm2 se tiene una resistencia DC de Ohm/km para un cable de cobre, se tiene que posee una resistencia de 0.0366 Ohm por lo que si se requiere de una corriente determinada para alimentar cada planta a 220V o 380V es necesario aplicar una regla de 3 compuesta para extraer el valor de la resistencia de pérdida a partir de https://www.gt-engineering.it/en/technical-standards/ul-and-nfpa-standards/ul-508a/cable-ampacity-ul-508a/, también es de anotar que es posible alimentar con un solo cable a partir de suma de corrientes, es decir, con corrientes menores por cada cable que se use para complementar la corriente objetivo a usar.

# PARA 220V

Grosor del cable sobre la Planta 1, a 220V:

In [88]:
print(f'La corriente necesaria es de {Corriente_Planta_1_220V_cambio} A para la planta 1, por lo que se necesita un solo cable que soporten 665 A de cobre, el cual corresponde a un valor de 2000 kcmil.')

R_km_P1_220_cambio = (500/1013)*0.0366
print(f'El grosor del cable necesario para 665 A es de 1013 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P1_220_cambio} Ohm')

Resistencia_Final_P1_220_cambio = (200*R_km_P1_220_cambio)/1000
print(f'Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P1_220_cambio} Ohm')

La corriente necesaria es de 654.1128222688013 A para la planta 1, por lo que se necesita un solo cable que soporten 665 A de cobre, el cual corresponde a un valor de 2000 kcmil.
El grosor del cable necesario para 665 A es de 1013 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.018065153010858835 Ohm
Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.003613030602171767 Ohm


In [89]:
Perdida_P1_220_cambio = Corriente_Planta_1_220V_Normal_cambio**2 * (Resistencia_Final_P1_220_cambio)
print(f'La pérdida asociada a la corriente a 220V para la planta 1 de una sola línea son de {Perdida_P1_220_cambio} W')
Perdida_P1_220_Tri_cambio = 3*Perdida_P1_220_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P1_220_Tri_cambio} W')

La pérdida asociada a la corriente a 220V para la planta 1 de una sola línea son de 989.3659030230248 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 2968.0977090690744 W


Grosor del cable sobre la Planta 2, a 220V:

In [91]:
print(f'La corriente necesaria es de {Corriente_Planta_2_220V_cambio} A para la planta 1, por lo que se necesita de dos cables que soporten 520 A de cobre, el cual corresponde a un valor de 900 kcmil.')

R_km_P2_220_cambio = (500/456)*0.0366
print(f'El grosor del cable necesario para 545 A es de 456 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P2_220_cambio} Ohm')

Resistencia_Final_P2_220_cambio = (300*R_km_P2_220_cambio)/1000
print(f'Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P2_220_cambio} Ohm')

La corriente necesaria es de 988.3047326449503 A para la planta 1, por lo que se necesita de dos cables que soporten 520 A de cobre, el cual corresponde a un valor de 900 kcmil.
El grosor del cable necesario para 545 A es de 456 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.04013157894736843 Ohm
Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.012039473684210529 Ohm


In [93]:
Paralelo_R_km_P2_220_cambio = Resistencia_Final_P2_220_cambio/2
Perdida_P2_220_cambio = Corriente_Planta_2_220V_Normal_cambio**2 * (Paralelo_R_km_P2_220_cambio)
print(f'La pérdida asociada a la corriente a 220V para la planta 2 son de {Perdida_P2_220_cambio} W')
Perdida_P2_220_Tri_cambio = 3*Perdida_P2_220_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P2_220_Tri_cambio} W')

La pérdida asociada a la corriente a 220V para la planta 2 son de 3763.0434264424944 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 11289.130279327483 W


Grosor del cable sobre la Planta 3, a 220V:

In [106]:
print(f'La corriente necesaria es de {Corriente_Planta_3_220V_cambio} A para la planta 1, por lo que se necesita de dos cables que soporten 590 A de cobre, el cual corresponde a un valor de 1250 kcmil.')

R_km_P3_220_cambio = (500/633)*0.0366
print(f'El grosor del cable necesario para 590 A es de 633 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P3_220_cambio} Ohm')

Resistencia_Final_P3_220_cambio = (350*R_km_P3_220_cambio)/1000
print(f'Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P3_220_cambio} Ohm')

La corriente necesaria es de 1128.8194722091384 A para la planta 1, por lo que se necesita de dos cables que soporten 590 A de cobre, el cual corresponde a un valor de 1250 kcmil.
El grosor del cable necesario para 590 A es de 633 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.02890995260663507 Ohm
Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.010118483412322275 Ohm


In [96]:
Paralelo_R_km_P3_220_cambio = Resistencia_Final_P3_220_cambio/2
Perdida_P3_220_cambio = Corriente_Planta_3_220V_Normal_cambio**2 * (Paralelo_R_km_P3_220_cambio)
print(f'La pérdida asociada a la corriente a 220V para la planta 3 son de {Perdida_P3_220_cambio} W')
Perdida_P3_220_Tri_cambio = 3*Perdida_P3_220_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P3_220_Tri_cambio} W')

La pérdida asociada a la corriente a 220V para la planta 3 son de 4125.859049539694 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 12377.577148619082 W


Grosor del cable sobre la Planta 4, a 220V:

In [105]:
print(f'La corriente necesaria es de {Corriente_Planta_4_220V_cambio} A para la planta 1, por lo que se necesita de dos cables que soporten 375 A de aluminio, el cual corresponde a un valor de 700 kcmil.')

R_km_P4_220_cambio = (500/355)*0.061
print(f'El grosor del cable necesario para 375 A es de 355 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P4_220_cambio} Ohm')

Resistencia_Final_P4_220_cambio = (300*R_km_P4_220_cambio)/1000
print(f'Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P4_220_cambio} Ohm')

La corriente necesaria es de 737.4429373393467 A para la planta 1, por lo que se necesita de dos cables que soporten 375 A de aluminio, el cual corresponde a un valor de 700 kcmil.
El grosor del cable necesario para 375 A es de 355 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.08591549295774648 Ohm
Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.025774647887323945 Ohm


In [99]:
Paralelo_R_km_P4_220_cambio = Resistencia_Final_P4_220_cambio/2
Perdida_P4_220_cambio = Corriente_Planta_4_220V_Normal_cambio**2 * (Paralelo_R_km_P4_220_cambio)
print(f'La pérdida asociada a la corriente a 220V para la planta 4 son de {Perdida_P4_220_cambio} W')
Perdida_P4_220_Tri_cambio = 3*Perdida_P4_220_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P4_220_Tri_cambio} W')

La pérdida asociada a la corriente a 220V para la planta 4 son de 4485.3832882117495 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 13456.149864635248 W


Grosor del cable sobre la Planta 5, a 220V:

In [104]:
print(f'La corriente necesaria es de {Corriente_Planta_5_220V_cambio} A para la planta 1, por lo que se necesita de tres cables que soporten 460 A de cobre, el cual corresponde a un valor de 700 kcmil.')

R_km_P5_220_cambio = (500/355)*0.0366
print(f'El grosor del cable necesario para 460 A es de 355 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P5_220_cambio} Ohm')

Resistencia_Final_P5_220_cambio = (500*R_km_P5_220_cambio)/1000
print(f'Ahora bien, la Planta 5 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P5_220_cambio} Ohm')

La corriente necesaria es de 1364.6728177373398 A para la planta 1, por lo que se necesita de tres cables que soporten 460 A de cobre, el cual corresponde a un valor de 700 kcmil.
El grosor del cable necesario para 460 A es de 355 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.05154929577464788 Ohm
Ahora bien, la Planta 5 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.02577464788732394 Ohm


In [102]:
Paralelo_R_km_P5_220_cambio = Resistencia_Final_P5_220_cambio/3
Perdida_P5_220_cambio = Corriente_Planta_5_220V_Normal_cambio**2 * (Paralelo_R_km_P5_220_cambio)
print(f'La pérdida asociada a la corriente a 220V para la planta 5 son de {Perdida_P5_220_cambio} W')
Perdida_P5_220_Tri_cambio = 3*Perdida_P5_220_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P5_220_Tri_cambio} W')

La pérdida asociada a la corriente a 220V para la planta 5 son de 10240.202444416127 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 30720.60733324838 W


# PARA 380V

Grosor del cable sobre la Planta 1, a 380V:

In [107]:
print(f'La corriente necesaria es de {Corriente_Planta_1_380V_cambio} A para la planta 1, por lo que se necesita de un solo cable que soporte 380 A de cobre, el cual corresponde a un valor de 500 kcmil.')

R_km_P1_380_cambio = (500/253)*0.0366
print(f'El grosor del cable necesario para 380 A es de 253 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P1_380_cambio} Ohm')

Resistencia_Final_P1_380_cambio = (200*R_km_P1_380_cambio)/1000
print(f'Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P1_380_cambio} Ohm')

La corriente necesaria es de 378.69689710299025 A para la planta 1, por lo que se necesita de un solo cable que soporte 380 A de cobre, el cual corresponde a un valor de 500 kcmil.
El grosor del cable necesario para 380 A es de 253 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.07233201581027668 Ohm
Ahora bien, la Planta 1 se encuentra a 200 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.014466403162055335 Ohm


In [110]:
Perdida_P1_380_cambio = Corriente_Planta_1_380V_Normal_cambio**2 * (Resistencia_Final_P1_380_cambio)
print(f'La pérdida asociada a la corriente a 380V para la planta 1 son de {Perdida_P1_380_cambio} W')
Perdida_P1_380_Tri_cambio = 3*Perdida_P1_380_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P1_380_Tri_cambio} W')

La pérdida asociada a la corriente a 380V para la planta 1 son de 1327.7736068150743 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 3983.320820445223 W


Grosor del cable sobre la Planta 2, a 380V:

In [113]:
print(f'La corriente necesaria es de {Corriente_Planta_2_380V_cambio} A para la planta 1, por lo que se necesita de un solo cable que soporte 590 A de cobre, el cual corresponde a un valor de 1250 kcmil.')

R_km_P2_380_cambio = (500/633)*0.0366
print(f'El grosor del cable necesario para 590 A es de 633 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P2_380_cambio} Ohm')

Resistencia_Final_P2_380_cambio = (300*R_km_P2_380_cambio)/1000
print(f'Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P2_380_cambio} Ohm')

La corriente necesaria es de 572.1764241628659 A para la planta 1, por lo que se necesita de un solo cable que soporte 590 A de cobre, el cual corresponde a un valor de 1250 kcmil.
El grosor del cable necesario para 590 A es de 633 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.02890995260663507 Ohm
Ahora bien, la Planta 2 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.00867298578199052 Ohm


In [114]:
Perdida_P2_380_cambio = Corriente_Planta_2_380V_Normal_cambio**2 * (Resistencia_Final_P2_380_cambio)
print(f'La pérdida asociada a la corriente a 380V para la planta 2 son de {Perdida_P2_380_cambio} W')
Perdida_P2_380_Tri_cambio = Perdida_P2_380_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P2_380_Tri_cambio} W')

La pérdida asociada a la corriente a 380V para la planta 2 son de 1817.224263804606 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 1817.224263804606 W


Grosor del cable sobre la Planta 3, a 380V:

In [116]:
print(f'La corriente necesaria es de {Corriente_Planta_3_380V_cambio} A para la planta 1, por lo que se necesita de un solo cable que soporte 665 A de cobre, el cual corresponde a un valor de 2000 kcmil.')

R_km_P3_380_cambio = (500/1013)*0.0366
print(f'El grosor del cable necesario para 665 A es de 1013 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P3_380_cambio} Ohm')

Resistencia_Final_P3_380_cambio = (350*R_km_P3_380_cambio)/1000
print(f'Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P3_380_cambio} Ohm')

La corriente necesaria es de 653.5270628579223 A para la planta 1, por lo que se necesita de un solo cable que soporte 665 A de cobre, el cual corresponde a un valor de 2000 kcmil.
El grosor del cable necesario para 665 A es de 1013 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.018065153010858835 Ohm
Ahora bien, la Planta 3 se encuentra a 350 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.006322803553800592 Ohm


In [117]:
Perdida_P3_380_cambio = Corriente_Planta_3_380V_Normal_cambio**2 * (Resistencia_Final_P3_380_cambio)
print(f'La pérdida asociada a la corriente a 220V para la planta 5 son de {Perdida_P3_380_cambio} W')
Perdida_P3_380_Tri_cambio = 3*Perdida_P3_380_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P3_380_Tri_cambio} W')

La pérdida asociada a la corriente a 220V para la planta 5 son de 1728.2907913544634 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 5184.87237406339 W


Grosor del cable sobre la Planta 4, a 380V:

In [120]:
print(f'La corriente necesaria es de {Corriente_Planta_4_380V_cambio} A para la planta 1, por lo que se necesita de un solo cable que soporte 445 A de aluminio, el cual corresponde a un valor de 1000 kcmil.')

R_km_P4_380_cambio = (500/506)*0.061
print(f'El grosor del cable necesario para 445 A es de 506 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P4_380_cambio} Ohm')

Resistencia_Final_P4_380_cambio = (300*R_km_P4_380_cambio)/1000
print(f'Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P4_380_cambio} Ohm')

La corriente necesaria es de 426.9406479333061 A para la planta 1, por lo que se necesita de un solo cable que soporte 445 A de aluminio, el cual corresponde a un valor de 1000 kcmil.
El grosor del cable necesario para 445 A es de 506 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.06027667984189723 Ohm
Ahora bien, la Planta 4 se encuentra a 300 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.01808300395256917 Ohm


In [121]:
Perdida_P4_380_cambio = Corriente_Planta_4_380V_Normal_cambio**2 * (Resistencia_Final_P4_380_cambio)
print(f'La pérdida asociada a la corriente a 380V para la planta 2 son de {Perdida_P4_380_cambio} W')
Perdida_P4_380_Tri_cambio = 3*Perdida_P4_380_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P4_380_Tri_cambio} W')

La pérdida asociada a la corriente a 380V para la planta 2 son de 2109.529295491614 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 6328.587886474842 W


Grosor del cable sobre la Planta 5, a 380V:

In [123]:
print(f'La corriente necesaria es de {Corriente_Planta_5_380V_cambio} A para la planta 1, por lo que se necesita de dos cables que soporte 420 A de cobre, el cual corresponde a un valor de 600 kcmil.')

R_km_P5_380_cambio = (500/304)*0.0366
print(f'El grosor del cable necesario para 420 A es de 304 mm\u00B2, por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de {R_km_P5_380_cambio} Ohm')

Resistencia_Final_P5_380_cambio = (500*R_km_P5_380_cambio)/1000
print(f'Ahora bien, la Planta 3 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a {Resistencia_Final_P5_380_cambio} Ohm')

La corriente necesaria es de 790.0737365847755 A para la planta 1, por lo que se necesita de dos cables que soporte 420 A de cobre, el cual corresponde a un valor de 600 kcmil.
El grosor del cable necesario para 420 A es de 304 mm², por lo cual la regla de 3 inversa necesaria para obtener la resistencia de Ohm/km es de 0.06019736842105263 Ohm
Ahora bien, la Planta 3 se encuentra a 500 metros del transformador, por lo cual la resistencia de pérdida será menor, esta equivale a 0.030098684210526316 Ohm


In [124]:
Paralelo_R_km_P5_380_cambio = Resistencia_Final_P5_380_cambio/2
Perdida_P5_380_cambio = Corriente_Planta_5_380V_Normal_cambio**2 * (Paralelo_R_km_P5_380_cambio)
print(f'La pérdida asociada a la corriente a 220V para la planta 5 son de {Perdida_P5_380_cambio} W')
Perdida_P5_380_Tri_cambio = 3*Perdida_P5_380_cambio
print(f'La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de {Perdida_P5_380_Tri_cambio} W')

La pérdida asociada a la corriente a 220V para la planta 5 son de 6012.190589005704 W
La pérdida asociada a la corrient e a 220V para la planta 1 para las 3 líneas son de 18036.571767017114 W


# POTENCIA A LA ENTRADA DEL TRANSFORMADOR AL CAMBIAR MOTORES

In [125]:
PerdidasXPlanta_Trifasica_220_cambio = [Perdida_P1_220_Tri_cambio, Perdida_P2_220_Tri_cambio, Perdida_P3_220_Tri_cambio, Perdida_P4_220_Tri_cambio, Perdida_P5_220_Tri_cambio]
PotenciaXPlanta_220_cambio = [Potencia_Planta_1_220_cambio, Potencia_Planta_2_220_cambio, Potencia_Planta_3_220_cambio, Potencia_Planta_4_220_cambio, Potencia_Planta_5_220_cambio]

PotenciaEntradaTransformador_sinAuxiliares_220_cambio = 0
Eficiencia_Transformador = 0.9
Potencia_Auxiliares = 480000

for i in range(len(PerdidasXPlanta_Trifasica_220_cambio)):
    PotenciaEntradaTransformador_sinAuxiliares_220_cambio += (PerdidasXPlanta_Trifasica_220_cambio[i] + PotenciaXPlanta_220_cambio[i])/Eficiencia_Transformador

PotenciaEntradaTransformador_220_cambio = PotenciaEntradaTransformador_sinAuxiliares_220_cambio + Potencia_Auxiliares

print(f'La potencia de entrada del transformador a 220V es de {PotenciaEntradaTransformador_220_cambio:,} W')

La potencia de entrada del transformador a 220V es de 1,511,690.7244466944 W


In [126]:
PerdidasXPlanta_Trifasica_380_cambio = [Perdida_P1_380_Tri_cambio, Perdida_P2_380_Tri_cambio, Perdida_P3_380_Tri_cambio, Perdida_P4_380_Tri_cambio, Perdida_P5_380_Tri_cambio]
PotenciaXPlanta_380_cambio = [Potencia_Planta_1_380_cambio, Potencia_Planta_2_380_cambio, Potencia_Planta_3_380_cambio, Potencia_Planta_4_380_cambio, Potencia_Planta_5_380_cambio]

PotenciaEntradaTransformador_sinAuxiliares_380_cambio = 0
Eficiencia_Transformador = 0.9
Potencia_Auxiliares = 480000

for i in range(len(PerdidasXPlanta_Trifasica_380_cambio)):
    PotenciaEntradaTransformador_sinAuxiliares_380_cambio += (PerdidasXPlanta_Trifasica_380_cambio[i] + PotenciaXPlanta_380_cambio[i])/Eficiencia_Transformador

PotenciaEntradaTransformador_380_cambio = PotenciaEntradaTransformador_sinAuxiliares_380_cambio + Potencia_Auxiliares

print(f'La potencia de entrada del transformador a 380V es de {PotenciaEntradaTransformador_380_cambio:,} W')

La potencia de entrada del transformador a 380V es de 1,472,289.6297543675 W


In [127]:
CostoTotal_220_13200_cambio = (PotenciaEntradaTransformador_220_cambio/1000) * TOTALHORAS * TARIFA_T1
CostoTotal_380_13200_cambio = (PotenciaEntradaTransformador_380_cambio/1000) * TOTALHORAS * TARIFA_T1
print(f'El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 13200/220 es de {CostoTotal_220_13200_cambio:,} pesos')
print(f'El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 13200/220 es de {CostoTotal_380_13200_cambio:,} pesos')

CostoTotal_220_44000_cambio = (PotenciaEntradaTransformador_220_cambio/1000) * TOTALHORAS * TARIFA_T2
CostoTotal_380_44000_cambio = (PotenciaEntradaTransformador_380_cambio/1000) * TOTALHORAS * TARIFA_T2
print(f'El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 44000/220 es de {CostoTotal_220_44000_cambio:,} pesos')
print(f'El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 44000/220 es de {CostoTotal_380_44000_cambio:,} pesos')

El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 13200/220 es de 6,183,903,964.049613 pesos
El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 13200/220 es de 6,022,725,105.361467 pesos
El costo total de la energía eléctrica consumida por los motores a 220V con un trasnformador de 44000/220 es de 5,399,764,588.874943 pesos
El costo total de la energía eléctrica consumida por los motores a 380V con un trasnformador de 44000/220 es de 5,259,023,739.942098 pesos
